In [ ]:
from torch import nn
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
class Vgg16_net(nn.Module):
    def __init__(self):
        super(Vgg16_net, self).__init__()

        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,stride=1,padding=1), #(32-3+2)/1+1=32   32*32*64
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1), #(32-3+2)/1+1=32    32*32*64
            
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=2,stride=2)   
        )

        self.layer2=nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),  #(16-3+2)/1+1=16  16*16*128
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=1), #(16-3+2)/1+1=16   16*16*128
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2,2)    
        )

        self.layer3=nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1),  #(8-3+2)/1+1=8   8*8*256
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),


            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1),  #(8-3+2)/1+1=8   8*8*256
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1),  #(8-3+2)/1+1=8   8*8*256
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2,2)     
        )

        self.layer4=nn.Sequential(
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1),  #(4-3+2)/1+1=4    4*4*512
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),   #(4-3+2)/1+1=4    4*4*512
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),   #(4-3+2)/1+1=4    4*4*512
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2,2)    
        )

        self.layer5=nn.Sequential(
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),   #(2-3+2)/1+1=2    2*2*512
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),  #(2-3+2)/1+1=2     2*2*512
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1),  #(2-3+2)/1+1=2      2*2*512
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2,2)   
        )


        self.conv=nn.Sequential(
            self.layer1,
            self.layer2,
            self.layer3,
            self.layer4,
            self.layer5
        )


    def forward(self,x):
        x=self.conv(x)
        
        return x


In [ ]:
vgg16=Vgg16_net()

In [ ]:
import torch
from torchvision import models
from torchsummary import summary
import pandas as pd
vgg16=Vgg16_net().to(device)
summary(vgg16,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
            Conv2d-2         [-1, 64, 224, 224]           1,792
       BatchNorm2d-3         [-1, 64, 224, 224]             128
       BatchNorm2d-4         [-1, 64, 224, 224]             128
              ReLU-5         [-1, 64, 224, 224]               0
              ReLU-6         [-1, 64, 224, 224]               0
            Conv2d-7         [-1, 64, 224, 224]          36,928
            Conv2d-8         [-1, 64, 224, 224]          36,928
       BatchNorm2d-9         [-1, 64, 224, 224]             128
      BatchNorm2d-10         [-1, 64, 224, 224]             128
             ReLU-11         [-1, 64, 224, 224]               0
             ReLU-12         [-1, 64, 224, 224]               0
        MaxPool2d-13         [-1, 64, 112, 112]               0
        MaxPool2d-14         [-1, 64, 1

In [ ]:
import torch
from torch import nn
class Bottleneck(nn.Module):

    extention=4
    def __init__(self,inplanes,planes,stride,downsample=None):
        
        super(Bottleneck, self).__init__()

        self.conv1=nn.Conv2d(inplanes,planes,kernel_size=1,stride=stride,bias=False)
        self.bn1=nn.BatchNorm2d(planes)

        self.conv2=nn.Conv2d(planes,planes,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(planes)

        self.conv3=nn.Conv2d(planes,planes*self.extention,kernel_size=1,stride=1,bias=False)
        self.bn3=nn.BatchNorm2d(planes*self.extention)

        self.relu=nn.ReLU(inplace=True)

        
        self.downsample=downsample
        self.stride=stride

    def forward(self,x):
        residual=x

        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)

        out=self.conv2(out)
        out=self.bn2(out)
        out=self.relu(out)

        out=self.conv3(out)
        out=self.bn3(out)
        out=self.relu(out)

        if self.downsample is not None:
            residual=self.downsample(x)

        out+=residual
        out=self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self,block,layers,num_class):
        self.inplane=64
        super(ResNet, self).__init__()

        self.block=block
        self.layers=layers

        self.conv1=nn.Conv2d(3,self.inplane,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(self.inplane)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.stage1=self.make_layer(self.block,64,layers[0],stride=1)
        self.stage2=self.make_layer(self.block,128,layers[1],stride=2)
        self.stage3=self.make_layer(self.block,256,layers[2],stride=2)
        self.stage4=self.make_layer(self.block,512,layers[3],stride=2)

        self.avgpool=nn.AvgPool2d(7)
        self.fc=nn.Linear(512*block.extention,num_class)

    def forward(self,x):
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        out=self.maxpool(out)

        out=self.stage1(out)
        out=self.stage2(out)
        out=self.stage3(out)
        out=self.stage4(out)

        return out

    def make_layer(self,block,plane,block_num,stride=1):
        block_list=[]
        downsample=None
        if(stride!=1 or self.inplane!=plane*block.extention):
            downsample=nn.Sequential(
                nn.Conv2d(self.inplane,plane*block.extention,stride=stride,kernel_size=1,bias=False),
                nn.BatchNorm2d(plane*block.extention)
            )

        conv_block=block(self.inplane,plane,stride=stride,downsample=downsample)
        block_list.append(conv_block)
        self.inplane=plane*block.extention

        for i in range(1,block_num):
            block_list.append(block(self.inplane,plane,stride=1))

        return nn.Sequential(*block_list)




resnet=ResNet(Bottleneck,[3,4,6,3],1000).to(device)



In [ ]:
summary(resnet,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
             ReLU-13          [-1, 256, 56, 56]               0
           Conv2d-14          [-1, 256,

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self,in_channel,out_channel,k,s,p,bias=False):
        super(ConvBlock,self).__init__()
        self.conv =nn.Sequential( nn.Conv2d(
        in_channel,out_channel,kernel_size=k,stride=s, padding=p,bias=bias),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(inplace=True),
        )
    def forward(self,x):
        x = self.conv(x)
        return x
class InceptionV3base(nn.Module):
    def __init__(self, in_channel,layers=[64,48,64,64,96,96,32] ):
        super(InceptionV3base,self).__init__()
        self.branch1 = nn.Sequential(
        nn.Conv2d(in_channel,layers[0],kernel_size=1,stride=1,padding=0 ,bias=False),
        nn.BatchNorm2d(layers[0]),
        nn.ReLU(inplace=True),
        )
        self.branch2 = nn.Sequential(
        nn.Conv2d(in_channel,layers[1],1,stride=1,padding=0,bias=False),
        nn.BatchNorm2d(layers[1]),
        nn.ReLU(inplace=True),
        nn.Conv2d(layers[1],layers[2],5,stride=1,padding=2,bias=False),
        nn.BatchNorm2d(layers[2]),
        nn.ReLU(True),
        )
        self.branch3 = nn.Sequential(
        nn.Conv2d(in_channel,layers[3],kernel_size=1,stride=1,padding=0,bias=False),
        nn.BatchNorm2d(layers[3]),
        nn.ReLU(inplace=True),
        nn.Conv2d(layers[3],layers[4],kernel_size=3,stride=1,padding=1,bias=False),
        nn.BatchNorm2d(layers[4]),
        nn.ReLU(inplace=True),
        nn.Conv2d(layers[4],layers[5],kernel_size=3,stride=1,padding=1,bias=False),
        nn.BatchNorm2d(layers[5]),
        nn.ReLU(inplace=True),
        )
        self.branch4 = nn.Sequential(
        nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
        nn.Conv2d(in_channel,layers[6],1,stride=1,padding=0,bias=False),
        nn.BatchNorm2d(layers[6]),
        nn.ReLU(inplace=True),
        )
    def forward(self,x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        return torch.cat([b1,b2,b3,b4],dim=1)
class InceptionV3_discount(nn.Module):
    def __init__(self,in_channel,layers=[384,64,96,96],block=ConvBlock):
        super(InceptionV3_discount,self).__init__()
        self.branch1 = nn.Sequential(
        block(in_channel,layers[0],3,2,0),
        )
        self.branch2 = nn.Sequential(
        block(in_channel,layers[1],1,1,0),
        block(layers[1],layers[2],3,1,1),
        block(layers[2],layers[3],3,2,0),
        )
        self.branch3 = nn.Sequential(
        nn.MaxPool2d(3,2,padding=0),
        )
    def forward(self,x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        return torch.cat([b1,b2,b3],dim=1)

class InceptionV3Block(nn.Module):
    def __init__(self,in_channel,layers=[192,128,128,192,128,128,128,128,192,192],block=ConvBlock):
        super(InceptionV3Block,self).__init__()
        self.branch1 = nn.Sequential(
        block(in_channel,layers[0],1,1,0),)
        self.branch2 = nn.Sequential(
        block(in_channel,layers[1],1,1,0),
        block(layers[1],layers[2],(1,7),1,(0,3)),
        block(layers[2],layers[3],(7,1),1,(3,0)),
        )
        self.branch3 = nn.Sequential(
            block(in_channel,layers[4],1,1,0),
            block(layers[4],layers[5],(7,1),1,(3,0)),
            block(layers[5],layers[6],(1,7),1,(0,3)),
            block(layers[6],layers[7],(7,1),1,(3,0)),
            block(layers[7],layers[8],(1,7),1,(0,3)),
        )
        self.branch4 = nn.Sequential(
        nn.AvgPool2d(3,1,padding=1),
        block(in_channel,layers[9],1,1,0),
        )
    def forward(self,x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        return torch.cat([b1,b2,b3,b4],dim=1)

class InceptionV3Substraction(nn.Module):
    def __init__(self,in_channel,layers=[192,320,192,192,192,192],block=ConvBlock):
        super(InceptionV3Substraction,self).__init__()
        self.branch1 = nn.Sequential(
        block(in_channel,layers[0],1,1,0),
        block(layers[0],layers[1],3,2,0),
        )
        self.branch2 = nn.Sequential(
        block(in_channel,layers[2],1,1,0),
        block(layers[2],layers[3],(1,7),1,(0,3)),
        block(layers[3],layers[4],(7,1),1,(3,0)),
        block(layers[4],layers[5],(3,3),2,0),
        )
        self.branch3 = nn.Sequential(
        nn.MaxPool2d(3,stride=2,padding=0),
        )
    def forward(self,x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        return torch.cat([b1,b2,b3],dim=1)
class Concat(nn.Module):
    def __init__(self,in_channel,layers=[384,384],block=ConvBlock):
        super(Concat,self).__init__()
        self.branch1 = nn.Sequential(
        block(in_channel,layers[0],(1,3),1,(0,1)),
        )
        self.branch2 = nn.Sequential(
        block(in_channel,layers[1],(3,1),1,(1,0)),
        )
    def forward(self,x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        return torch.cat([b1,b2],dim=1)

class InceptionLastLayer(nn.Module):
    def __init__(self,in_channel,layers=[320,384,384,448,384,384,384,192],block=ConvBlock,concat=Concat):
        super(InceptionLastLayer,self).__init__()
        self.branch1 = nn.Sequential(
        block(in_channel,layers[0],1,1,0),
        )
        self.branch2 = nn.Sequential(
        block(in_channel,layers[1],1,1,0),
        block(layers[1],layers[2],(1,3),1,(0,1)),
        block(layers[2],layers[3],(3,1),1,(1,0)),
        )
        self.branch3 = nn.Sequential(
        block(in_channel,layers[4],(3,3),1,1),
        concat(layers[4],[layers[5],layers[6]] ),
        )
        self.branch4 = nn.Sequential(
        nn.AvgPool2d(3,1,padding=1),
        block(in_channel,layers[7],1,1,0),)
    def forward(self,x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        return torch.cat([b1,b2,b3,b4],dim=1)


Inceptionv3_shape = 224
class InceptionV3(nn.Module):
    def __init__(self,num_class,block=ConvBlock,base=InceptionV3base,dicount=InceptionV3_discount,base7block=InceptionV3Block,
    Substraction=InceptionV3Substraction,lastblock=InceptionLastLayer):
        super(InceptionV3,self).__init__()
        self.bottle = nn.Sequential(
        block(3,32,3,2,0),
        block(32,32,3,1,0),
        block(32,64,3,1,1),
        nn.MaxPool2d(3,2,padding=0),
        block(64,80,1,1,0),
        block(80,192,3,1,0),
        nn.MaxPool2d(3,2,padding=0),
        )

        self.layer1 = nn.Sequential(base(192))
        self.layer2 = nn.Sequential(base(256,[64,48,64,64,96,96,64]))
        self.layer3 = nn.Sequential(base(288,[64,48,64,64,96,96,64]))
        self.layer4 = nn.Sequential(dicount(288))
        self.layer5 = nn.Sequential(base7block(768))
        self.layer6 = nn.Sequential(base7block(768,[192,160,160,192,160,160,160,160,192,192]))
        self.layer7 = nn.Sequential(base7block(768,[192,160,160,192,160,160,160,160,192,192]))
        self.layer8 = nn.Sequential(base7block(768,[192,192,192,192,192,192,192,192,192,192]))
        self.layer9 = nn.Sequential(Substraction(768))
        self.layer10 = nn.Sequential(
        lastblock(1280),)
        self.layer11 = nn.Sequential(lastblock(1728))
        self.avg = nn.AvgPool2d(8,stride=1,padding=0)
        self.fc = nn.Sequential(nn.Dropout(p=0.8),nn.Linear(1728,num_class),)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif  isinstance(m,nn.Linear):
                m.weight.data.normal_(0,0.001)
                m.bias.data.zero_()
    def forward(self,x):
        x = self.bottle(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x =self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x =self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x =self.layer9(x)
        x =self.layer10(x)
        x = self.layer11(x)
        x = self.avg(x)
        return x

In [ ]:
IncV3 = InceptionV3(2).to(device)
summary(IncV3,(3,299,299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
              ReLU-3         [-1, 32, 149, 149]               0
         ConvBlock-4         [-1, 32, 149, 149]               0
            Conv2d-5         [-1, 32, 147, 147]           9,216
       BatchNorm2d-6         [-1, 32, 147, 147]              64
              ReLU-7         [-1, 32, 147, 147]               0
         ConvBlock-8         [-1, 32, 147, 147]               0
            Conv2d-9         [-1, 64, 147, 147]          18,432
      BatchNorm2d-10         [-1, 64, 147, 147]             128
             ReLU-11         [-1, 64, 147, 147]               0
        ConvBlock-12         [-1, 64, 147, 147]               0
        MaxPool2d-13           [-1, 64, 73, 73]               0
           Conv2d-14           [-1, 80,

In [ ]:
#lstm

In [ ]:
layer=nn.LSTM(input_size=10,
            hidden_size=3,
            num_layers=1,
            batch_first=True)

In [ ]:
layer.state_dict()[]['weight_ih_l0'

tensor([[ 0.0654, -0.3733, -0.0991,  0.1919, -0.4678, -0.3454,  0.5222,  0.3522,
         -0.0755, -0.3313],
        [-0.3000,  0.3332, -0.5111,  0.1441, -0.2760,  0.0533, -0.0831,  0.2240,
          0.2663, -0.3477],
        [-0.2603, -0.2722, -0.1456,  0.1814,  0.0043, -0.1016,  0.1311, -0.0484,
          0.4457, -0.2364],
        [ 0.1712,  0.4973,  0.4262, -0.2991, -0.1670, -0.3681, -0.5424,  0.4758,
         -0.2123,  0.5017],
        [ 0.2426, -0.5633, -0.1255,  0.2236, -0.5099,  0.1872, -0.0861,  0.1386,
         -0.1111,  0.2494],
        [ 0.3052,  0.5150,  0.0707, -0.1595,  0.2952, -0.1881, -0.2803,  0.0662,
          0.5578, -0.1490],
        [-0.4484, -0.0893, -0.2692,  0.3828, -0.1572, -0.3873,  0.4426, -0.2759,
         -0.3151,  0.5624],
        [ 0.3864,  0.4037, -0.0150, -0.1108, -0.1606,  0.0961, -0.3214, -0.2753,
         -0.0631,  0.4536],
        [ 0.0504, -0.0093,  0.0967, -0.0810,  0.2669,  0.3659,  0.3039, -0.2347,
         -0.3754,  0.3021],
        [-0.0837, -

In [ ]:
[layer.state_dict()[i].shape for i in list(layer.state_dict())]

[torch.Size([12, 10]), torch.Size([12, 3]), torch.Size([12]), torch.Size([12])]

In [ ]:
import numpy as np
X=np.array([[84345856,29446272],[41495552,23508032],[39112400,27452896]])
y=np.array([190.71,193.87,191.56])

In [ ]:
from sklearn.linear_model import LinearRegression as LR
lr=LR().fit(X,y)

In [ ]:
lr.coef_

array([ 6.83204197e-09, -5.81444178e-07])

In [ ]:
lr.intercept_

207.255108988361